In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float', "{:.4f}".format)

In [11]:
train = pd.read_csv('data/TrainAndValid.csv', low_memory=False)
test = pd.read_csv('data/test.csv', low_memory=False)

In [12]:
train['saledate'] = pd.to_datetime(train.saledate)
test['saledate'] = pd.to_datetime(test.saledate)

In [13]:
train.set_index('saledate', inplace=True)
train.sort_index(inplace=True)
train.index.name = None

test.set_index('saledate', inplace=True)
test.sort_index(inplace=True)
test.index.name = None

In [16]:
# This will convert object type into category
for label, content in train.items():
    if pd.api.types.is_string_dtype(content):
        train[label] = content.astype("category").cat.as_ordered()
        
for label, content in test.items():
    if pd.api.types.is_string_dtype(content):
        test[label] = content.astype("category").cat.as_ordered()

In [17]:
for label, content in train.items():
    if pd.api.types.is_numeric_dtype(content) and pd.isnull(content).sum():
        # Add a column that shows if the data was missing
        train[label + '_missing'] = pd.isnull(content)
        train[label].fillna(content.median(), inplace=True)
        
for label, content in test.items():
    if pd.api.types.is_numeric_dtype(content) and pd.isnull(content).sum():
        # Add a column that shows if the data was missing
        test[label + '_missing'] = pd.isnull(content)
        test[label].fillna(content.median(), inplace=True)

In [19]:
for label, content in train.items():
    if not pd.api.types.is_numeric_dtype(content):
        train[label + '_missing'] = pd.isnull(content)
        train[label] = pd.Categorical(content).codes + 1

for label, content in test.items():
    if not pd.api.types.is_numeric_dtype(content):
        test[label + '_missing'] = pd.isnull(content)
        test[label] = pd.Categorical(content).codes + 1

In [21]:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

0
0
